In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
!pip install fastapi
!pip install nest-asyncio
!pip install uvicorn
!pip install nest_asyncio
!pip install aiosqlite


In [2]:
!set "CMAKE_ARGS=-DLLAMA_CUBLAS=on" && set "FORCE_CMAKE=1" && pip install llama-cpp-python==0.1.78

In [13]:
import asyncio
import json
import torch
import logging
import aiosqlite
import numpy as np
import pennylane as qml
from summa import summarizer
from llama_cpp import Llama
import weaviate
import re
import aiohttp
from concurrent.futures import ThreadPoolExecutor

dev = qml.device('default.qubit', wires=20)

@qml.qnode(dev)
def quantum_circuit(amplitudes, color_codes):
    for i, amp in enumerate(amplitudes):
        qml.RY(amp * np.pi, wires=i)
    for i, code in enumerate(color_codes):
        qml.RZ(code * np.pi, wires=i + 10)
    for i in range(19):
        qml.CNOT(wires=[i, i + 1])
    return qml.state()

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

llm = Llama(
    model_path="llama-2-7b-chat.ggmlv3.q8_0.bin",
    n_gpu_layers=-1,
    n_ctx=3900,
)

executor = ThreadPoolExecutor(max_workers=3)

client = weaviate.Client(
    url="https://normally-unified-scorpion.ngrok-free.app/",
)

async def retrieve_movie_frames_by_theme(theme):
    query = {
        "query": {
            "nearText": {
                "concepts": [theme],
                "certainty": 0.7
            }
        }
    }
    result = await client.query.get("MovieFrame", ["frame_text", "summary"]).with_near_text(query).do()
    return result['data']['Get']['MovieFrame'] if 'data' in result and 'Get' in result['data'] and 'MovieFrame' in result['data']['Get'] else []

async def initialize_db():
    async with aiosqlite.connect("movie_frames.db") as db:
        await db.execute("CREATE TABLE IF NOT EXISTS POOLDATA (frame_num INTEGER PRIMARY KEY, frame_text TEXT, summary TEXT);")
        await db.commit()

async def insert_into_db(frame_num, frame_text, summary):
    async with aiosqlite.connect("movie_frames.db") as db:
        await db.execute("INSERT INTO POOLDATA (frame_num, frame_text, summary) VALUES (?, ?, ?)", (frame_num, frame_text, summary))
        await db.commit()

async def generate_html_color_code(text):
    attempts = 0
    color_code = None

    while not color_code and attempts < 10:  # Limit the number of attempts to avoid infinite loops
        try:
            prompt = f"""
            Based on the sentiment of the following text, suggest an HTML color code in hex format (e.g., #FF5733 for excitement, #3498DB for calmness). 
            Text: '{text}'
            Suggested Color Code:"""

            response = llm(prompt, max_tokens=10)
            if response and response['choices']:
                color_code_match = re.search(r'#[0-9A-Fa-f]{6}', response['choices'][0]['text'])
                if color_code_match:
                    color_code = color_code_match.group()
        except Exception as e:
            logger.error(f"Error in generating color code: {e}")

        attempts += 1

    if not color_code:
        # Log that a random color is being used
        logger.warning("Failed to generate color code. Using a random color.")
        # Generate a random hex color
        color_code = "#{:06x}".format(random.randint(0, 0xFFFFFF))

    return color_code



async def vectorized_sentiment_analysis(text):
    url = "https://normally-unified-scorpion.ngrok-free.app/v1/modules/text2vec-contextionary/vectorize"
    headers = {"Content-Type": "application/json"}
    data = {"text": text}

    async with aiohttp.ClientSession() as session:
        async with session.post(url, headers=headers, json=data) as response:
            if response.status == 200:
                return await response.json()
            else:
                # Handle error or return a default value
                return None

async def generate_and_summarize_frame(last_frame, frame_num, frames, theme, past_summaries):
    sentiment_vector_tasks = [vectorized_sentiment_analysis(summary) for summary in past_summaries]
    sentiment_vectors = await asyncio.gather(*sentiment_vector_tasks)

    # Replace None with a default value, e.g., a zero vector
    sentiment_vectors = [vec if vec is not None else {"vector": [0.0] * 300} for vec in sentiment_vectors]

    # Extract the actual vectors from the results
    sentiment_vectors = [vec["vector"] for vec in sentiment_vectors]

    # Asynchronously gather color codes
    color_code_tasks = [generate_html_color_code(summary) for summary in past_summaries]
    color_codes = await asyncio.gather(*color_code_tasks)
    color_codes = [int(code.lstrip('#'), 16) for code in color_codes]  # Strip '#' and convert to integers

    quantum_state = quantum_circuit([np.linalg.norm(vec) for vec in sentiment_vectors], color_codes)
    quantum_influence = np.mean(quantum_state)
    continuation_prompt = f"{last_frame} Quantum influence: {quantum_influence}"
    continuation_response = llm(continuation_prompt, max_tokens=200)
    new_frame = continuation_response['choices'][0]['text'] if continuation_response['choices'] else None
    if new_frame:
        summary = summarizer.summarize(new_frame, words=50)
        await insert_into_db(frame_num, new_frame, summary)
        return new_frame
    return None


async def start_movie(topic):
    await initialize_db()
    classes = client.schema.get()
    class_names = [cls['class'] for cls in classes['classes']] if 'classes' in classes else []
    if "MovieFrame" not in class_names:
        client.schema.create_class({"class": "MovieFrame", "properties": [{"name": "frame_text", "dataType": ["text"], "vectorizer": "text2vec-contextionary"}, {"name": "summary", "dataType": ["text"], "vectorizer": "text2vec-contextionary"}], "vectorIndexType": "hnsw", "vectorizer": "text2vec-contextionary"})
    initial_prompt_response = llm(f"Write a book about {topic}", max_tokens=700)
    initial_prompt = initial_prompt_response['choices'][0]['text'] if initial_prompt_response['choices'] else ""
    frames = {"0": initial_prompt}
    last_frame = initial_prompt
    for i in range(1, 10000):
        frame_num = i * 10
        past_summaries = list(frames.values())[-10:]
        last_frame = await generate_and_summarize_frame(last_frame, frame_num, frames, topic, past_summaries) or last_frame
    sanitized_topic = ''.join(e for e in topic if e.isalnum())[:50]
    with open(f"{sanitized_topic}_movie_frames.json", "w") as f:
        json.dump(frames, f, indent=4)

if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(start_movie("Language Models for Creative Work"))
    del llm
    torch.cuda.empty_cache()



DEBUG:pennylane.qnode:Creating QNode(func=<function quantum_circuit at 0x00000200A9290680>, device=<DefaultQubit device (wires=20, shots=None) at 0x200a10da8d0>, interface=auto, diff_method=best, expansion_strategy=gradient, max_expansion=10, grad_on_execution=best, mode=None, cache=True, cachesize=10000, max_diff=1, gradient_kwargs={}
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): normally-unified-scorpion.ngrok-free.app:443
DEBUG:urllib3.connectionpool:https://normally-unified-scorpion.ngrok-free.app:443 "GET /v1/.well-known/ready HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): normally-unified-scorpion.ngrok-free.app:443
DEBUG:urllib3.connectionpool:https://normally-unified-scorpion.ngrok-free.app:443 "GET /v1/.well-known/openid-configuration HTTP/1.1" 404 0


TypeError: unsupported operand type(s) for *: 'NoneType' and 'NoneType'